In [1]:
import pandas as pd
import holidays
from datetime import date, timedelta
import os

def gerar_dim_datas(ano_inicio, ano_fim, nome_arquivo='dim_datas.csv'):
    """
    Gera uma tabela dimensão de datas com feriados brasileiros e salva em CSV.
    
    Args:
        ano_inicio (int): Ano de início da série histórica.
        ano_fim (int): Ano final da série histórica.
        nome_arquivo (str): Caminho/nome do arquivo de saída.
    """
    
    print(f"--- Iniciando geração da Dimensão Data ({ano_inicio}-{ano_fim}) ---\n")
    
    # 1. Criar o range de datas
    start_date = date(ano_inicio, 1, 1)
    end_date = date(ano_fim, 12, 31)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # 2. Inicializar o DataFrame
    df = pd.DataFrame(date_range, columns=['data'])
    
    # 3. Configurar Feriados Brasil (Nacional)
    # Obs: É possível adicionar estado/município: holidays.Brazil(subdiv='SP')
    rs_holidays = holidays.Brazil(subdiv='RS') 
    
    # 4. Engenharia de Atributos (Extração de componentes da data)
    
    # Básico
    df['id_data'] = df['data'].dt.strftime('%Y%m%d').astype(int) # Chave primária integer
    df['ano'] = df['data'].dt.year
    df['mes'] = df['data'].dt.month
    df['dia'] = df['data'].dt.day
    
    # Trimestre e Semestre
    df['trimestre'] = df['data'].dt.quarter
    df['semestre'] = (df['mes'] - 1) // 6 + 1
    
    # 5. Tradução e Formatação (PT-BR)
    
    # Mapeamento de Dias da Semana
    map_dias_semana = {
        0: 'Segunda-feira', 1: 'Terça-feira', 2: 'Quarta-feira',
        3: 'Quinta-feira', 4: 'Sexta-feira', 5: 'Sábado', 6: 'Domingo'
    }
    map_dias_abrev = {
        0: 'Seg', 1: 'Ter', 2: 'Qua', 3: 'Qui', 4: 'Sex', 5: 'Sab', 6: 'Dom'
    }
    
    df['dia_semana_num'] = df['data'].dt.dayofweek # 0=Segunda, 6=Domingo
    df['nome_dia_semana'] = df['dia_semana_num'].map(map_dias_semana)
    df['nome_dia_abrev'] = df['dia_semana_num'].map(map_dias_abrev)
    
    # Mapeamento de Meses
    map_meses = {
        1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril',
        5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
        9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
    }
    map_meses_abrev = {
        1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Abr', 5: 'Mai', 6: 'Jun',
        7: 'Jul', 8: 'Ago', 9: 'Set', 10: 'Out', 11: 'Nov', 12: 'Dez'
    }
    
    df['nome_mes'] = df['mes'].map(map_meses)
    df['nome_mes_abrev'] = df['mes'].map(map_meses_abrev)
    
    # 6. Lógica de Feriados e Dias Úteis
    
    # Função auxiliar para verificar feriado
    def check_holiday(x):
        return x in rs_holidays

    # Função auxiliar para pegar nome do feriado
    def get_holiday_name(x):
        return rs_holidays.get(x)

    # Aplicação
    df['eh_feriado'] = df['data'].apply(check_holiday)
    df['nome_feriado'] = df['data'].apply(get_holiday_name)
    
    # Final de semana (Sábado=5, Domingo=6)
    df['eh_final_semana'] = df['dia_semana_num'].isin([5, 6])
    
    # Dia Útil (Não é final de semana E não é feriado)
    # Nota: Em algumas regras de negócio, sábado pode ser útil. Ajuste conforme necessidade.
    df['eh_dia_util'] = (~df['eh_final_semana']) & (~df['eh_feriado'])
    
    # Formatação Brasileira de Data (dd/mm/aaaa) para visualização
    df['data_pt_br'] = df['data'].dt.strftime('%d/%m/%Y')

    # 7. Organização das Colunas
    cols_order = [
        'id_data', 'data', 'data_pt_br', 'ano', 'mes', 'dia',
        'nome_mes', 'nome_mes_abrev', 'trimestre', 'semestre',
        'dia_semana_num', 'nome_dia_semana', 'nome_dia_abrev',
        'eh_final_semana', 'eh_feriado', 'nome_feriado', 'eh_dia_util'
    ]
    df = df[cols_order]
    
    # 8. Exportação
    try:
        df.to_csv(nome_arquivo, index=False, encoding='utf-8-sig')
        print(f"Sucesso! Arquivo gerado em: {os.path.abspath(nome_arquivo)}")
        print(f"Total de registros: {len(df)}")
    except Exception as e:
        print(f"Erro ao salvar arquivo: {e}")

    return df



In [2]:
if __name__ == "__main__":
    # Configure aqui o período desejado
    ANO_INICIAL = 2020
    ANO_FINAL = 2025
    
    df_result = gerar_dim_datas(ANO_INICIAL, ANO_FINAL)
    
    # Exibe as primeiras linhas para validação
    print("\nVisualização das primeiras 5 linhas:")
    print(df_result[['data_pt_br', 'nome_dia_semana', 'nome_feriado', 'eh_dia_util']].head())

--- Iniciando geração da Dimensão Data (2020-2025) ---

Sucesso! Arquivo gerado em: d:\Projetos\MC_Supermercados\dim_datas.csv
Total de registros: 2192

Visualização das primeiras 5 linhas:
   data_pt_br nome_dia_semana                nome_feriado  eh_dia_util
0  01/01/2020    Quarta-feira  Confraternização Universal        False
1  02/01/2020    Quinta-feira                        None         True
2  03/01/2020     Sexta-feira                        None         True
3  04/01/2020          Sábado                        None        False
4  05/01/2020         Domingo                        None        False
